In [1]:
# 05_two_models_router_debug.py
import os
import uuid
import json
import logging
from datetime import datetime
from dotenv import load_dotenv

load_dotenv()

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

# ---------- Logging setup ----------
logger = logging.getLogger("two_model_router")
logger.setLevel(logging.INFO)
handler = logging.StreamHandler()
handler.setFormatter(logging.Formatter("%(asctime)s %(levelname)s %(message)s"))
logger.handlers = [handler]

def jlog(event: str, **fields):
    payload = {
        "ts": datetime.utcnow().isoformat() + "Z",
        "event": event,
        **fields,
    }
    logger.info(json.dumps(payload, ensure_ascii=False))

def safe_route(text: str) -> str:
    t = (text or "").strip().upper()
    return "SIMPLE" if t == "SIMPLE" else "COMPLEX"

def main():
    # Model A (fast) does routing + simple answers
    fast = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)
    # Model B (smart) does complex answers
    smart = ChatOpenAI(model="gpt-4.1", temperature=0.2)  # change if needed

    router_prompt = ChatPromptTemplate.from_messages([
        ("system",
         "You are a router. Decide if the user's request is SIMPLE or COMPLEX.\n"
         "- SIMPLE: short, direct Q&A, definitions, formatting, light reasoning.\n"
         "- COMPLEX: multi-step reasoning, planning, code, long explanations, high accuracy needs.\n"
         "Reply with exactly one word: SIMPLE or COMPLEX."),
        ("user", "{input}")
    ])

    simple_prompt = ChatPromptTemplate.from_messages([
        ("system", "You are fast and concise. Answer clearly in <= 8 sentences."),
        ("user", "{input}")
    ])

    complex_prompt = ChatPromptTemplate.from_messages([
        ("system", "You are thorough and careful. Provide structured output and verify assumptions."),
        ("user", "{input}")
    ])

    print("Ask something. Type exit to quit.")
    session_id = os.getenv("SESSION_ID", "local-dev")

    while True:
        user_input = input("\nAsk> ").strip()
        if not user_input:
            continue
        if user_input.lower() in {"exit", "quit"}:
            break

        request_id = str(uuid.uuid4())[:8]
        jlog("request.start", session_id=session_id, request_id=request_id, input=user_input)

        try:
            # ---- Router decision ----
            router_resp = (router_prompt | fast).invoke({"input": user_input})
            route_raw = router_resp.content.strip()
            route = safe_route(route_raw)

            jlog("route.decision",
                 session_id=session_id,
                 request_id=request_id,
                 route_raw=route_raw,
                 route=route,
                 router_model="gpt-4o-mini")

            # ---- Execute with chosen model ----
            if route == "SIMPLE":
                exec_model = "gpt-4o-mini"
                resp = (simple_prompt | fast).invoke({"input": user_input})
            else:
                exec_model = "gpt-4.1"
                resp = (complex_prompt | smart).invoke({"input": user_input})

            output = resp.content

            # Optional: capture response metadata if present (varies by wrapper)
            meta = getattr(resp, "response_metadata", None)

            jlog("request.end",
                 session_id=session_id,
                 request_id=request_id,
                 route=route,
                 exec_model=exec_model,
                 output=output,
                 response_metadata=meta)

            print(f"\n[Route={route} | Model={exec_model}]\n{output}")

        except Exception as e:
            jlog("request.error", session_id=session_id, request_id=request_id, error=str(e))
            print("\nError:", e)

if __name__ == "__main__":
    main()

Ask something. Type exit to quit.


/var/folders/vk/j_r2f1rs58q7c4bywftwhkgm0000gn/T/ipykernel_91649/2086120054.py:23: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "ts": datetime.utcnow().isoformat() + "Z",
2026-02-18 20:25:40,046 INFO {"ts": "2026-02-19T02:25:40.046567Z", "event": "request.start", "session_id": "local-dev", "request_id": "449667cb", "input": "Search Ai and also let me know the courses and tell me the best authors and give me a lunch menu ingredients and a study plan with the lunch in between"}
2026-02-18 20:25:40,590 INFO {"ts": "2026-02-19T02:25:40.590536Z", "event": "route.decision", "session_id": "local-dev", "request_id": "449667cb", "route_raw": "COMPLEX", "route": "COMPLEX", "router_model": "gpt-4o-mini"}
2026-02-18 20:25:57,015 INFO {"ts": "2026-02-19T02:25:57.014924Z", "event": "request.end", "session_id": "local-dev", "request_id": "4496


[Route=COMPLEX | Model=gpt-4.1]
Certainly! Here’s a structured response to your multi-part request:

---

## 1. **Search: AI Overview**

**Artificial Intelligence (AI)** is a branch of computer science focused on building smart machines capable of performing tasks that typically require human intelligence. These tasks include learning, reasoning, problem-solving, perception, and language understanding.

**Key Areas in AI:**
- Machine Learning (ML)
- Deep Learning (DL)
- Natural Language Processing (NLP)
- Computer Vision
- Robotics
- Expert Systems

---

## 2. **AI Courses**

### **Top Online AI Courses (2024):**

| Course Name | Platform | Level | Description |
|-------------|----------|-------|-------------|
| AI For Everyone | Coursera (Andrew Ng) | Beginner | Non-technical intro to AI concepts and societal impact. |
| Machine Learning | Coursera (Andrew Ng) | Beginner/Intermediate | Foundational ML algorithms and techniques. |
| Deep Learning Specialization | Coursera (Andrew Ng) 